In [59]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
import time
import numpy as np

In [60]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def clean_column(df,lst_column):
    """
    Функция для очистки от колонки от пустых значений
    """
    for column in lst_column:
        df[column].fillna('Отсутствует',inplace=True)
        df[column] = df[column].astype(str) # Приводим к строковому виду
        df[column] = df[column].apply(lambda x:x.replace('nan','Отсутствует'))
        df[column] = df[column].apply(lambda x:x.strip()) # очищаем от пробельных символов(на случай если в ячейке стоит просто пробел или несколько пробелов)
        df[column] = df[column].apply(lambda x:x if x else 'Отсутствует') # Если пустая строка то заменяем на значение Не заполнено

def prepare_column_to_int(cell):
    """
    Функция для обработки ячеек в инт, чтобы они в экселе потом не отображались как строки
    """
    try:
        float_value = float(cell)
        int_value = int(float_value)
        return int_value
    except ValueError:
        return cell

In [61]:
# file_data_xlsx = 'data/Реестр УПП.xlsx'
file_data_xlsx = 'data/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'
path_to_end_folder = 'data/'

In [62]:
use_cols = list(range(25))

In [63]:
df = pd.read_excel(file_data_xlsx,skiprows=8,usecols=use_cols)

In [64]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [65]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
Площадь лесотаксационного выдела или его части (лесопатологического выдела), га - 8
Год лесоустройства -11



"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\nПлощадь лесотаксационного выдела или его части (лесопатологического выдела), га - 8\nГод лесоустройства -11\n\n\n\n'

In [66]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га','8':'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га',
                  '11':'Год лесоустройства'},inplace=True)

In [67]:
df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",7,"Площадь лесотаксационного выдела или его части (лесопатологического выдела), га",9,10,...,16,17,18,19,20,21,22,23,24,25
0,Алданское,Алданское,NaN,21,20,74.0,NaN,1.3,NaN,NaN,...,21.0,09.01.-30.04.2020,869.0,Л,более 40%,более 40%,21.12.2020,1.0,0.0,"2011, ИФЛ 2020 года"
1,Алданское,Алданское,NaN,21,20,74.0,NaN,2.9,NaN,NaN,...,21.0,09.01.-30.04.2020,869.0,Л,более 40%,более 40%,21.12.2020,1.0,0.0,"2011, ИФЛ 2020 года"
2,Алданское,Алданское,NaN,21,21,31.0,NaN,4.2,NaN,NaN,...,21.0,09.01.-30.04.2020,869.0,Л,более 40%,более 40%,21.12.2020,1.0,0.0,"2011, ИФЛ 2020 года"
3,Алданское,Алданское,NaN,22,13,39.0,NaN,5.2,NaN,NaN,...,21.0,09.01.-30.04.2020,869.0,Л,более 40%,более 40%,21.12.2020,1.0,0.0,"2011, ИФЛ 2020 года"
4,Алданское,Алданское,NaN,22,15,67.0,NaN,8.5,NaN,NaN,...,21.0,09.01.-30.04.2020,869.0,Л,более 40%,более 40%,21.12.2020,1.0,0.0,"2011, ИФЛ 2020 года"


In [68]:
# заполняем пропущенные места
clean_column(df,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Год лесоустройства'])

In [69]:
region = 1

In [70]:
# Бурятия
if region == 0:
    # СОздаем проверочный файл для проверки правильности ввода плошади выдела
    check_df = df.copy(deep=True)

    # Меняем тип столбца на строку чтобы создать строку включающую в себя все значения разделенные ;
    check_df['Площадь лесотаксационного выдела, га'] = check_df['Площадь лесотаксационного выдела, га'].astype(str)
    #

    checked_pl = check_df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела']).agg(
        {'Площадь лесотаксационного выдела, га': combine})

    # Применяем функцию првоеряющую количество уникальных значений в столбце, если больше одного то значит есть ошибка в данных
    checked_pl['Контроль совпадения площади выдела'] = checked_pl['Площадь лесотаксационного выдела, га'].apply(
        check_unique)

    # переименовывам колонку
    checked_pl.rename(columns={'Площадь лесотаксационного выдела, га': 'Все значения площади для указанного выдела'},
                      inplace=True)
    # Извлекаем индексы в колонки
    checked_pl = checked_pl.reset_index()
    
        # конвертируем в инт чтобы корректно отображалось
    checked_pl['Номер лесного квартала'] = checked_pl['Номер лесного квартала'].apply(prepare_column_to_int)
    checked_pl['Номер лесотаксационного выдела'] = checked_pl['Номер лесотаксационного выдела'].apply(prepare_column_to_int)
   
   
    # Сортируем
    checked_pl.sort_values(by=['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела'],inplace=True)
    

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = openpyxl.Workbook() # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(checked_pl,index=False,header=True):
        wb['Sheet'].append(row)

    # Форматирование итоговой таблицы
    # Ширина колонок
    wb['Sheet'].column_dimensions['A'].width = 15
    wb['Sheet'].column_dimensions['B'].width = 20
    wb['Sheet'].column_dimensions['C'].width = 36
    wb['Sheet'].column_dimensions['F'].width = 15
    wb['Sheet'].column_dimensions['G'].width = 15
    # Перенос строк для заголовков
    wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)


    wb.save(f'{path_to_end_folder}/Бурятия Проверка правильности ввода площадей лесотаксационного выдела {current_time}.xlsx')




    # Основной отчет
    # Готовим колонки к группировке
    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: x.replace(',', '.'))

    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].astype(str)

    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(
        lambda x: x.replace(',', '.'))


    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(convert_to_float)



    # Группируем
    group_df = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                           'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                                   'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'sum'})

    group_df=group_df.astype(str)

    # переименовываем колонку
    group_df.rename(columns={
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'Используемая площадь лесотаксационного выдела, га'},
                    inplace=True)

    # Извлекаем индексы в колонки
    group_df = group_df.reset_index()

    group_df['Площадь лесотаксационного выдела, га'] = group_df['Площадь лесотаксационного выдела, га'].apply(convert_to_float)
    group_df['Используемая площадь лесотаксационного выдела, га'] =group_df['Используемая площадь лесотаксационного выдела, га'].apply(convert_to_float)

    # Округляем до 3 знаков для корректного сравнения
    group_df['Площадь лесотаксационного выдела, га'] = np.round(group_df['Площадь лесотаксационного выдела, га'],decimals=3)
    group_df['Используемая площадь лесотаксационного выдела, га'] = np.round(group_df['Используемая площадь лесотаксационного выдела, га'],decimals=3)




    # Создаем колонку для контроля 
    group_df['Контроль площади используемого надела'] = group_df['Площадь лесотаксационного выдела, га'] < group_df[
        'Используемая площадь лесотаксационного выдела, га']

    group_df['Контроль площади используемого надела'] = group_df['Контроль площади используемого надела'].apply(
        lambda x: 'Превышение используемой площади выдела!!!' if x is True else 'Все в порядке')

    # Изменяем состояние колонки если площадь всего выдела равна 0
    group_df['Контроль правильности ввода площади лесотаксационного выдела'] = group_df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: 'Площадь лесотаксационного выдела равна нулю или  обнаружены разные значения площади выдела !!!' if x == 0 else 'Площади лесотаксационного выдела не отличаются друг от друга')


    
            # конвертируем в инт чтобы корректно отображалось
    group_df['Номер лесного квартала'] = group_df['Номер лесного квартала'].apply(prepare_column_to_int)
    group_df['Номер лесотаксационного выдела'] = group_df['Номер лесотаксационного выдела'].apply(prepare_column_to_int)
   
   
    # Сортируем
    group_df.sort_values(by=['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела'],inplace=True)

    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = openpyxl.Workbook() # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(group_df,index=False,header=True):
        wb['Sheet'].append(row)

    # Форматирование итоговой таблицы
    # Ширина колонок
    wb['Sheet'].column_dimensions['A'].width = 15
    wb['Sheet'].column_dimensions['B'].width = 20
    wb['Sheet'].column_dimensions['C'].width = 36
    wb['Sheet'].column_dimensions['F'].width = 15
    wb['Sheet'].column_dimensions['G'].width = 15
    wb['Sheet'].column_dimensions['H'].width = 20
    # Перенос строк для заголовков
    wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)

    wb.save(f'{path_to_end_folder}/Бурятия Контроль используемых площадей лесотаксационных выделов от {current_time}.xlsx')


#Якутия
elif region == 1:
    # СОздаем проверочный файл для проверки правильности ввода плошади выдела
    check_df = df.copy(deep=True)

    # Меняем тип столбца на строку чтобы создать строку включающую в себя все значения разделенные ;
    check_df['Площадь лесотаксационного выдела, га'] = check_df['Площадь лесотаксационного выдела, га'].astype(str)
    
        # Заполняем год лесоустройства на случай если он не заполнен или заполнен пробелами
    df['Год лесоустройства'] = df['Год лесоустройства'].astype(str)
    df['Год лесоустройства'] = df['Год лесоустройства'].apply(lambda x:x.replace(' ','Отсутствует'))
    df['Год лесоустройства'] = df['Год лесоустройства'].apply(lambda x:x.replace('nan','Отсутствует'))
    
    checked_pl = check_df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела','Год лесоустройства']).agg(
        {'Площадь лесотаксационного выдела, га': combine})

    # Применяем функцию првоеряющую количество уникальных значений в столбце, если больше одного то значит есть ошибка в данных
    checked_pl['Контроль совпадения площади выдела'] = checked_pl['Площадь лесотаксационного выдела, га'].apply(
        check_unique)

    # переименовывам колонку
    checked_pl.rename(columns={'Площадь лесотаксационного выдела, га': 'Все значения площади для указанного выдела'},
                      inplace=True)
    # Извлекаем индексы в колонки
    checked_pl = checked_pl.reset_index()
    
            # конвертируем в инт чтобы корректно отображалось
    checked_pl['Номер лесного квартала'] = checked_pl['Номер лесного квартала'].apply(prepare_column_to_int)
    checked_pl['Номер лесотаксационного выдела'] = checked_pl['Номер лесотаксационного выдела'].apply(prepare_column_to_int)
    checked_pl['Год лесоустройства'] = checked_pl['Год лесоустройства'].apply(prepare_column_to_int)
   
   
    # Сортируем
    checked_pl.sort_values(by=['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела','Год лесоустройства'],inplace=True)
    

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = openpyxl.Workbook() # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(checked_pl,index=False,header=True):
        wb['Sheet'].append(row)

    # Форматирование итоговой таблицы
    # Ширина колонок
    wb['Sheet'].column_dimensions['A'].width = 15
    wb['Sheet'].column_dimensions['B'].width = 20
    wb['Sheet'].column_dimensions['C'].width = 36
    wb['Sheet'].column_dimensions['F'].width = 15
    wb['Sheet'].column_dimensions['G'].width = 15
    # Перенос строк для заголовков
    wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)


    wb.save(f'{path_to_end_folder}/Саха(Якутия) Проверка правильности ввода площадей лесотаксационного выдела {current_time}.xlsx')
    
    
        # Основной отчет
    # Готовим колонки к группировке
    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: x.replace(',', '.'))


    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].astype(str)

    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(
        lambda x: x.replace(',', '.'))


    df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(convert_to_float)





    # Группируем
    group_df = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                           'Номер лесотаксационного выдела','Год лесоустройства']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                                   'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'sum'})


    group_df=group_df.astype(str)

    # переименовываем колонку
    group_df.rename(columns={
        'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'Используемая площадь лесотаксационного выдела, га'},
                    inplace=True)

    # Извлекаем индексы в колонки
    group_df = group_df.reset_index()

    group_df['Площадь лесотаксационного выдела, га'] = group_df['Площадь лесотаксационного выдела, га'].apply(convert_to_float)
    group_df['Используемая площадь лесотаксационного выдела, га'] =group_df['Используемая площадь лесотаксационного выдела, га'].apply(convert_to_float)



    group_df['Площадь лесотаксационного выдела, га'] = group_df['Площадь лесотаксационного выдела, га'].apply(convert_to_float)
    group_df['Используемая площадь лесотаксационного выдела, га'] =group_df['Используемая площадь лесотаксационного выдела, га'].apply(convert_to_float)

    # Округляем до 3 знаков для корректного сравнения
    group_df['Площадь лесотаксационного выдела, га'] = np.round(group_df['Площадь лесотаксационного выдела, га'],decimals=3)
    group_df['Используемая площадь лесотаксационного выдела, га'] = np.round(group_df['Используемая площадь лесотаксационного выдела, га'],decimals=3)




    # Создаем колонку для контроля 
    group_df['Контроль площади используемого надела'] = group_df['Площадь лесотаксационного выдела, га'] < group_df[
        'Используемая площадь лесотаксационного выдела, га']

    group_df['Контроль площади используемого надела'] = group_df['Контроль площади используемого надела'].apply(
        lambda x: 'Превышение используемой площади выдела!!!' if x is True else 'Все в порядке')

    # Изменяем состояние колонки если площадь всего выдела равна 0
    group_df['Контроль правильности ввода площади лесотаксационного выдела'] = group_df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: 'Площадь лесотаксационного выдела равна нулю или  обнаружены разные значения площади выдела !!!' if x == 0 else 'Площади лесотаксационного выдела не отличаются друг от друга')


                # конвертируем в инт чтобы корректно отображалось
    group_df['Номер лесного квартала'] = group_df['Номер лесного квартала'].apply(prepare_column_to_int)
    group_df['Номер лесотаксационного выдела'] = group_df['Номер лесотаксационного выдела'].apply(prepare_column_to_int)
    group_df['Год лесоустройства'] = group_df['Год лесоустройства'].apply(prepare_column_to_int)
   
   
    # Сортируем
    group_df.sort_values(by=['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                                   'Номер лесотаксационного выдела','Год лесоустройства'],inplace=True)
    
    
    
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = openpyxl.Workbook() # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(group_df,index=False,header=True):
        wb['Sheet'].append(row)

    # Форматирование итоговой таблицы
    # Ширина колонок
    wb['Sheet'].column_dimensions['A'].width = 15
    wb['Sheet'].column_dimensions['B'].width = 20
    wb['Sheet'].column_dimensions['C'].width = 36
    wb['Sheet'].column_dimensions['F'].width = 15
    wb['Sheet'].column_dimensions['G'].width = 15
    wb['Sheet'].column_dimensions['H'].width = 20
    # Перенос строк для заголовков
    wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
    wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)

    wb.save(f'{path_to_end_folder}/Саха(Якутия) Контроль используемых площадей лесотаксационных выделов от {current_time}.xlsx')

    